<a href="https://colab.research.google.com/github/Hamoda-dabbit/Mining---classification-in-Arabic-Article/blob/main/DecisionTree/DecisionTree_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Classification by DecisionTree and Count Vectorizer**

----------

 import library

  إستيراد المكتبات

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


--------

**1- Import Data set**

In [2]:
# df = pd.read_excel("drive/MyDrive/mining/1.xlsx")
# df = pd.read_excel("drive/MyDrive/mining/2.xlsx")
# df = pd.read_excel("drive/MyDrive/mining/3.xlsx")
df = pd.read_excel("drive/MyDrive/mining/4.xlsx")

------
shuffle  
بعثرة البيانات

In [3]:
df=shuffle(df)
df

,text,targe
33213,أكد امحمد فاخر مدرب فريق الرجاء البيضاوي أن ال...,4
61273,أعلن بنك المغرب أن صافي الاحتياطي من العملات ا...,2
656,توفي الممثل المصري وائل نور مساء الأربعاء الما...,0
67177,شهد عام تألق لافت لبعض النجوم الكبار فوق الثلا...,4
13135,ذكرت منظمة الشرطة الجنائية الدولية الإنتربول أ...,4
...,...,...
45918,أكد رئيس مجلس النواب الحبيب المالكي أنه تابع ب...,3
19491,تجسيدا لقيم التسامح لدى الشباب المغربي ومعه شب...,0
22490,لفظ شخص في الستينيات من العمر أنفاسه الأخيرة ب...,1
9060,أكد رئيس الجمعية المغربية لصناعة وتسويق السيار...,2


------
**2- Clean Data**

تنظيف البيانات

In [4]:
print(np.sum(df.isnull().any(axis=1)))

clean_df= df.dropna()
print(np.sum(clean_df.isnull().any(axis=1)))
#-------------
print (df.notnull().any(axis = 0))
print(len(df))
#-------------
def remove_hashtag(df, col = 'text'):
    for letter in r'#.][!XR':
      df[col] = df[col].astype(str).str.replace(letter,'', regex=True)
        
remove_hashtag(clean_df)
clean_df
#-------------
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
#-------------
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text
#-------------    
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)
#-------------
def processDocument(doc, stemmer): 

    #Replace @username with empty string
    doc = re.sub(r'@[^\s]+', ' ', doc)
    doc = re.sub(r'_', ' ', doc)
    doc = re.sub(r'\n', ' ', doc)
    doc = re.sub(r'[a-z,A-Z]', '', doc)
    doc = re.sub(r'\d', '', doc)
    #Convert www.* or https?://* to " "
    doc = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',doc)
    #Replace #word with word
    doc = re.sub(r'#([^\s]+)', r'\1', doc)
    # remove punctuations
    doc= remove_punctuations(doc)
    # normalize the tweet
    doc= normalize_arabic(doc)
    # remove repeated letters
    doc=remove_repeating_char(doc)
    #stemming
    doc = stemmer.stem(doc)
    
    return doc

stemmer = ISRIStemmer()
clean_df["text"] = clean_df['text'].apply(lambda x: processDocument(x, stemmer))
print(clean_df)

378
0
text     True
targe    True
dtype: bool
68690
                                                    text  targe
33213  اكد امحمد فاخر مدرب فريق الرجاء البيضاوي ان ال...      4
61273  اعلن بنك المغرب ان صافي الاحتياطي من العملات ا...      2
656    توفي المثل المصري وائل نور مساء الاربعاء الماض...      0
67177  شهد عام تالق لافت لبعض النجوم الكبار فوق الثلا...      4
13135  ذكرت منظمه الشرطه الجنائيه الدوليه الانتربول ا...      4
...                                                  ...    ...
45918  اكد رئيس مجلس النواب الحبيب المالكي انه تابع ب...      3
19491  تجسيدا لقيم التسامح لدي الشباب المغربي ومعه شب...      0
22490  لفظ شخص في الستينيات من العمر انفاسه الاخيره ب...      1
9060   اكد رئيس الجمعيه المغربيه لصناعه وتسويق السيار...      2
36983  اعلنت وزاره الثقافه ان الدعم الاجمالي المقدم ل...      0

[68312 rows x 2 columns]


---------
accuracy calculation function

عمل تابع لحساب الدقة

In [5]:
def calculate_results(y_true, y_pred):
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

----------------------
**3- Feature Extraction:** Count Vectorizer


**4- classification:** Decision Tree


استخراج الميزات(تحويل النص إلى أرقام)


with Cross validation

لتوزيع عينات الاختبار والتأكد من عدم وجود

Overfitting

In [6]:
%%time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn import tree
X =clean_df['text'].values
y = clean_df['targe'].values
vectorizer = CountVectorizer(max_features=1500)

strtfdKFold = StratifiedKFold(n_splits=5)
kfold = strtfdKFold.split(X, y)
clf = tree.DecisionTreeClassifier(random_state=5)
result_DecisionTreeClassifier=[]
#
for k, (train_index, test_index) in enumerate(kfold):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    vectorizer.fit(X_train)
    X_train = vectorizer.transform(X_train)
    X_test  = vectorizer.transform(X_test)
    model = clf.fit(X_train, y_train)
    y_pred=(model.predict(X_test))
    result_DecisionTreeClassifier.append(calculate_results(y_test, y_pred))


CPU times: user 7min 8s, sys: 1.26 s, total: 7min 9s
Wall time: 7min 11s


حساب متوسط النتائج للـ5 مراحل في cross validation 

In [7]:
accuracy=0
f1=0
precision=0
recall=0
for i in result_DecisionTreeClassifier:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(result_DecisionTreeClassifier)
f1=f1/len(result_DecisionTreeClassifier)
precision=precision/len(result_DecisionTreeClassifier)
recall=recall/len(result_DecisionTreeClassifier)
# (result_DecisionTreeClassifier)
results_DecisionTreeClassifier=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]
results_DecisionTreeClassifier

[{'accuracy': 84.60737963985387,
  'f1': 0.8462427239222862,
  'precision': 0.8464992140097001,
  'recall': 0.846073796398539}]

--------
-----
---------
**5- Show Results**

النتائج

نتائج التدريب لكل قاعدة بيانات

In [ ]:
print('DecisionTreeClassifier results in dataset 1:')
results_DecisionTreeClassifier

DecisionTreeClassifier results in dataset 1:


[{'accuracy': 87.03992020073217,
  'f1': 0.8704465840666646,
  'precision': 0.8706003060169341,
  'recall': 0.8703992020073219}]

In [ ]:
print('DecisionTreeClassifier results in dataset 2:')
results_DecisionTreeClassifier

DecisionTreeClassifier results in dataset 2:


[{'accuracy': 86.33242710541131,
  'f1': 0.86338687648086,
  'precision': 0.863583486718646,
  'recall': 0.8633242710541131}]

In [ ]:
print('DecisionTreeClassifier results in dataset 3:')
results_DecisionTreeClassifier

DecisionTreeClassifier results in dataset 3:


[{'accuracy': 85.1399435546625,
  'f1': 0.8515250417175342,
  'precision': 0.8517474993200912,
  'recall': 0.8513994355466249}]

In [8]:
print('DecisionTreeClassifier results in dataset 4:')
results_DecisionTreeClassifier

DecisionTreeClassifier results in dataset 4:


[{'accuracy': 84.60737963985387,
  'f1': 0.8462427239222862,
  'precision': 0.8464992140097001,
  'recall': 0.846073796398539}]

In [9]:
Final_Result=[
  {'accuracy': 87.03992020073217,
  'f1': 0.8704465840666646,
  'precision': 0.8706003060169341,
  'recall': 0.8703992020073219}
    ,
  {'accuracy': 86.33242710541131,
  'f1': 0.86338687648086,
  'precision': 0.863583486718646,
  'recall': 0.8633242710541131}
  ,
  {'accuracy': 85.1399435546625,
  'f1': 0.8515250417175342,
  'precision': 0.8517474993200912,
  'recall': 0.8513994355466249}
  ,
  {'accuracy': 84.60737963985387,
  'f1': 0.8462427239222862,
  'precision': 0.8464992140097001,
  'recall': 0.846073796398539}
  ]

accuracy=0
f1=0
precision=0
recall=0
for i in Final_Result:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(Final_Result)
f1=f1/len(Final_Result)
precision=precision/len(Final_Result)
recall=recall/len(Final_Result)
# (result_DecisionTreeClassifier)
Final_Result=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]
print("RAM:  1.6 GB ,  CPU times: user 7min 8s ")
print('Final_Result:')
(Final_Result)

RAM:  1.6 GB ,  CPU times: user 7min 8s 
Final_Result:


[{'accuracy': 85.77991762516496,
  'f1': 0.8579003065468364,
  'precision': 0.8581076265163428,
  'recall': 0.8577991762516497}]